In [ ]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 12.3 MB/s 
     |████████████████████████████████| 3.3 MB 65.5 MB/s 
     |████████████████████████████████| 636 kB 83.7 MB/s 
     |████████████████████████████████| 895 kB 62.7 MB/s 
     |████████████████████████████████| 56 kB 4.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BertTokenizer, BertModel, AutoModel, AutoTokenizer
import pandas as pd
import numpy as np
import torch

In [ ]:
#preprocessing:entry to tokens, map each token to integers
from transformers import AutoTokenizer
#checkpoint = "giacomomiolo/electramed_base_scivocab_1M"
#checkpoint = "google/electra-base-discriminator"
checkpoint = "dmis-lab/biobert-base-cased-v1.2"


model = BertModel.from_pretrained(checkpoint, output_hidden_states = True,)
tokenizer = BertTokenizer.from_pretrained(checkpoint)

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.2 were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def bert_text_preparation(text, tokenizer):
    """Preparing the input for BERT
    
    Takes a string argument and performs
    pre-processing like adding special tokens,
    tokenization, tokens to ids, and tokens to
    segment ids. All tokens are mapped to seg-
    ment id = 1.
    
    Args:
        text (str): Text to be converted
        tokenizer (obj): Tokenizer object
            to convert text into BERT-re-
            adable tokens and ids
        
    Returns:
        list: List of BERT-readable tokens
        obj: Torch tensor with token ids
        obj: Torch tensor segment ids
    
    
    """
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1]*len(indexed_tokens)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    return tokenized_text, tokens_tensor, segments_tensors
    
def get_bert_embeddings(tokens_tensor, segments_tensors, model):
    """Get embeddings from an embedding model
    
    Args:
        tokens_tensor (obj): Torch tensor size [n_tokens]
            with token ids for each token in text
        segments_tensors (obj): Torch tensor size [n_tokens]
            with segment ids for each token in text
        model (obj): Embedding model to generate embeddings
            from token and segment ids
    
    Returns:
        list: List of list of floats of size
            [n_tokens, n_embedding_dimensions]
            containing embeddings for each token
    
    """
    
    # Gradient calculation id disabled
    # Model is in inference mode
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        # Removing the first hidden state
        # The first state is the input state
        hidden_states = outputs[2][1:]

    # Getting embeddings from the final BERT layer
    token_embeddings = hidden_states[-1]
    # Collapsing the tensor into 1-dimension
    token_embeddings = torch.squeeze(token_embeddings, dim=0)
    # Converting torchtensors to lists
    list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]

    return list_token_embeddings

In [ ]:
from google.colab import files

df=pd.read_csv("./Abstracts-Parsed.csv", sep=',')
#df=pd.read_csv("./Methods-Parsed.csv", sep=',')
df=df.rename(columns={'Content_Parsed': 'text'})
df=df.drop(columns={'Unnamed: 0'})

df.head()

,File_Name,Content,Category,text,Category_Code
0,Gapped Blast,Gapped BLAST and PSI-BLAST: a new generation o...,Alignment,gap blast psiblast new generation protein d...,0
1,RapSearch,RAPSearch: a fast protein similarity search to...,Alignment,rapsearch fast protein similarity search tool...,0
2,PhenoMeter,PhenoMeter: A Metabolome Database Search Tool ...,Alignment,phenometer metabolome database search tool us...,0
3,cuBlASTp,cuBLASTP: Fine-Grained Parallelization of Prot...,Alignment,cublastp finegrained parallelization protein ...,0
4,muBLASTP,muBLASTP: database-indexed protein sequence se...,Alignment,mublastp databaseindexed protein sequence sear...,0


In [ ]:
import unicodedata
pd.options.mode.chained_assignment = None  # default='warn'

import re
re.compile('<title>(.*)</title>')
for i in range (len(df['text'])):
  df['text'][i] = unicodedata.normalize('NFKD', df['text'][i]).encode('ascii', 'ignore').decode("utf-8")
  df['text'][i] = re.sub(r'[^\w]', ' ', df['text'][i])
  df['text'][i] = df['text'][i].encode("ascii", "ignore")
  df['text'][i] = df['text'][i].decode()  

In [ ]:
# Getting embeddings for the target

embeddings = []
all_embeddings = []
sentence_embedding = np.empty(768, dtype=object)
c=0
for text in df["text"]:
    print("*****************item ",c)
    tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(text, tokenizer)
    list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)
    embeddings.append(list_token_embeddings)
    c=c+1

*****************item  0
*****************item  1
*****************item  2
*****************item  3
*****************item  4
*****************item  5
*****************item  6
*****************item  7
*****************item  8
*****************item  9
*****************item  10
*****************item  11
*****************item  12
*****************item  13
*****************item  14
*****************item  15
*****************item  16
*****************item  17
*****************item  18
*****************item  19
*****************item  20
*****************item  21
*****************item  22
*****************item  23
*****************item  24
*****************item  25
*****************item  26
*****************item  27
*****************item  28
*****************item  29
*****************item  30
*****************item  31
*****************item  32
*****************item  33
*****************item  34
*****************item  35
*****************item  36
*****************item  37
*****************item 

In [ ]:
for l in range(len(embeddings)):
  for v in embeddings[l]:
    sentence_embedding=np.vstack((sentence_embedding, v))
  sentence_embedding = np.delete(sentence_embedding, obj=0, axis=0)
  sentence_embedding = (np.mean(sentence_embedding, axis=0)).tolist()
  all_embeddings.append(sentence_embedding)

all_embeddings = np.array(all_embeddings)
all_embeddings = pd.DataFrame(all_embeddings)

all_embeddings.insert(loc=0, column='text', value=df['text'])

In [ ]:
#all_embeddings.to_csv('AbstractsMethodsROBERTAstsb.csv') 
#files.download("AbstractsMethodsROBERTAstsb.csv")
all_embeddings.to_csv('Abstracts-BioBERT-NoSL.csv') 
files.download("Abstracts-BioBERT-NoSL.csv")